In [1]:
# external imports
import os
import scipy
import numpy as np
import matplotlib.pyplot as plt

# setup for plots
plt.rcParams["text.usetex"] = True
plt.rc("font", family="serif")
plt.rc("xtick", labelsize="x-small")
plt.rc("ytick", labelsize="x-small")
plt.rc("text", usetex=True)

### Information 

This notebook reproduces the results from Bonacci et al. [1] 
by using a copy of the functions defined in *src/utils/yambo_gw_conv_class.py* that 
are used for the NPJ/SOTA convergence workflows. 

### Structure

We copy the parameters for the fitting grids shown in the supplementary information [2] for each given material.
Then we parse the gaps for each grid from the provided raw data files (*raw_input_output* [3]). Then the functions *get_best_fit* and *suggest_next_point* are called to check if the next point matches the one shown in the corresponding figure [2]. Then the convergence criteria are checked. This is repeated until the last point is found.

### Material hBN

[1] https://doi.org/10.1038/s41524-023-01027-2 \
[2] https://static-content.springer.com/esm/art%3A10.1038%2Fs41524-023-01027-2/MediaObjects/41524_2023_1027_MOESM1_ESM.pdf \
[3] https://archive.materialscloud.org/record/2022.161

In [2]:
# material 
mat = "bulk_hBN"

In [3]:
# settings (from the paper and *aiida-yambo* github)
conv_thr = 0.082
conv_grad = 5e-5
conv_hessian = 1e-8
bnd_max = 1600 # random value...
cut_max = 36 
bnd_step = 200
cut_step = 4
step = [bnd_step, cut_step]

In [4]:
# grids from supplementary figure 1
param_grids = []
param_grids.append(np.array([[200,4], [200,16], [400,12], [600,8], [800,4], [800,16]]))
param_grids.append(param_grids[0] + step)

In [5]:
# function to parse the direct gap from the output file
def get_direct_gw_gap(f_name):
    data = np.loadtxt(f_name, comments="#")
    return (data[1, 2] + data[1, 3]) - (data[0, 2] + data[0, 3])

In [6]:
# parse the calculated gaps
qp_file_name = "o-aiida.out.qp"
mat_path = os.path.join("raw_input_output", mat)
grid_num = 0
grid_plot = []
grids = []
for g in param_grids:
    temp_grid = []
    for p in g:
        temp_gap = get_direct_gw_gap(os.path.join(mat_path, f"{p[0]:d}_{p[1]:d}_12X12X4", qp_file_name))
        grid_plot.append([p[0], p[1], temp_gap, grid_num])
        temp_grid.append([p[0], p[1], temp_gap])
    grids.append(np.array(temp_grid))
    grid_num += 1
grid_plot = np.array(grid_plot)

The inputs and return statements of the functions were adjusted because we work with them outside the intended class...

In [7]:
def fit2d(grid, conv_grad, conv_hessian, alpha=1, beta=1):
    # fit functions for the convergence surface
    f = lambda x, a, b, c, d: (a / (x[0] ** alpha) + b) * (c / (x[1] ** beta) + d)

    # wrong definitions of the fit function derivatives from the aiida yambo repository
    """
    fx = lambda x, a, c, d: (
        (-alpha * a / (x[0] ** (alpha + 1))) * (c / (x[1]) + d)
    )
    fy = lambda x, a, b, c: (
        (a / (x[0]) + b) * (-beta * c / (x[1] ** (beta + 1)))
    )
    """

    # corrected definitions of the fit function derivatives
    fx = lambda x, a, c, d: (
        (-alpha * a / (x[0] ** (alpha + 1))) * (c / (x[1] ** beta) + d)
    )
    fy = lambda x, a, b, c: (
        (a / (x[0] ** alpha) + b) * (-beta * c / (x[1] ** (beta + 1)))
    )

    # off-diagonal elements of the hessian matrix of the fit function
    fxy = lambda x, a, c: (
        (-alpha * a / (x[0] ** (alpha + 1))) * (-beta * c / (x[1] ** (beta + 1)))
    )

    # fit data
    xdata, ydata = (
        np.array((grid[:, 0], grid[:, 1])),
        grid[:, 2],
    )

    # fit the convergence surface
    popt, _ = scipy.optimize.curve_fit(
        f,
        xdata=xdata,
        ydata=ydata,
        sigma=1 / (xdata[0] * xdata[1]),
        bounds=(
            [-np.inf, -np.inf, -np.inf, -np.inf],
            [np.inf, np.inf, np.inf, np.inf],
        ),
    )

    # convergence critierum
    mae_int = np.average(
        (
            abs(
                f(
                    xdata,
                    popt[0],
                    popt[1],
                    popt[2],
                    popt[3],
                )
                - ydata
            )
        ),
        weights=xdata[0] * xdata[1],
    )
    mae_fit = mae_int

    # get the extrapolated gap value
    extra = popt[1] * popt[3]

    # new grid to extrapolate the function far from the starting grid
    x_fit = np.arange(min(xdata[0]), max(xdata[0]) * 10, bnd_step)
    y_fit = np.arange(min(xdata[1]), max(xdata[1]) * 10, cut_step)

    # obtain the derivatives
    zx_fit = fx(np.meshgrid(x_fit, y_fit), popt[0], popt[2], popt[3])
    zy_fit = fy(np.meshgrid(x_fit, y_fit), popt[0], popt[1], popt[2])
    zxy_fit = fxy(np.meshgrid(x_fit, y_fit), popt[0], popt[2])

    # get the fit function values on the new grid
    x_fit, y_fit = np.meshgrid(x_fit, y_fit)
    z_fit = f(
        np.meshgrid(x_fit, y_fit), popt[0], popt[1], popt[2], popt[3]
    )

    # check where the convergence conditions for the derivatives are meet
    condition_conv_calc = np.where(
        (np.abs(zx_fit) < conv_grad)
        & (np.abs(zy_fit) < conv_grad)
        & (np.abs(zxy_fit) < conv_hessian)
    )

    # save this fit to the old fit variable for later
    old_x_fit = x_fit
    old_y_fit = y_fit
    old_z_fit = z_fit

    # if no points match the convergence critieria return
    print(
        "Number of points that fullfil the derivative criteria: "
        + f"{len(x_fit[condition_conv_calc]):d}",
        flush=True,
    )

    if len(x_fit[condition_conv_calc]) == 0:
        return False, mae_fit, x_fit, y_fit, z_fit, extra

    # obtain a new grid where the estimated converged direct gap is situated
    b = max(max(xdata[0]), x_fit[condition_conv_calc][0] * 1.25)
    g = max(max(xdata[1]), y_fit[condition_conv_calc][0] * 1.25)

    # fit on the new grid with derivatives
    x_fit = np.arange(min(xdata[0]), b + 1, bnd_step)
    y_fit = np.arange(min(xdata[1]), g + 1, cut_step)
    z_fit = f(
        np.meshgrid(x_fit, y_fit), popt[0], popt[1], popt[2], popt[3]
    )
    zx_fit = fx(np.meshgrid(x_fit, y_fit), popt[0], popt[2], popt[3])
    zy_fit = fy(np.meshgrid(x_fit, y_fit), popt[0], popt[1], popt[2])
    zxy_fit = fxy(np.meshgrid(x_fit, y_fit), popt[0], popt[2])
    x_fit, y_fit = np.meshgrid(x_fit, y_fit)

    return True, mae_fit, x_fit, y_fit, z_fit, extra

def get_best_fit(grid, conv_grad, conv_hessian, power_laws=[1, 2]):
    # find the best initial fit parameters for the exponents
    error = 10  # random initial value
    for i in power_laws:
        for j in power_laws:
            print(f"\nTrying: alpha = {i:.2f}, beta = {j:.2f}", flush=True)
            fit_flag, mae_fit, x_fit, y_fit, z_fit, extra = fit2d(
                grid,
                conv_grad,
                conv_hessian,
                alpha=i,
                beta=j,
            )
            print(f"MAE = {mae_fit:.6f}", flush=True)
            if mae_fit < error:
                ii, jj = i, j
                error = mae_fit
    print(f"\nBest power law: alpha = {ii:.2f}, beta = {jj:.2f}\n", flush=True)

    # get the best initial fit
    fit_flag, mae_fit, x_fit, y_fit, z_fit, extra = fit2d(
        grid,
        conv_grad,
        conv_hessian,
        alpha=ii,
        beta=jj,
    )
    
    return fit_flag, mae_fit, x_fit, y_fit, z_fit, extra, ii, jj

def suggest_next_point(x_fit, y_fit, z_fit, extra, conv_thr, conv_percent=0):
    # reference
    reference = z_fit[-1, -1]
    print(
        "Extrapolated gap value from fit parameters: " + f"{extra:.6f} eV",
        flush=True,
    )
    print(
        "Extrapolated gap value at points that fulfill the derivative criteria: "
        + f"{reference:.6f} eV",
        flush=True,
    )

    # converge with percentages?
    if conv_percent > 0:
        conv_thr = conv_percent / 100 * reference

    # find a point that satisfies the convergence condition
    discrepancy = np.round(
        abs(reference - z_fit),
        abs(int(np.round(np.log10(conv_thr), 0))),
    )
    condition = np.where((discrepancy <= conv_thr))
    next_bnd_x, next_cutsrc = (
        x_fit[condition][0],
        y_fit[condition][0],
    )
    ref_gap = z_fit[condition][0]

    # dictionary that contains the relevant information for the next step
    next_step = {
        "Nb": next_bnd_x,
        "Gc": next_cutsrc,
        "gap": ref_gap,
        "ref_gap": reference,
        "new_grid": False,
        "already_computed": False,
        "conv_thr": conv_thr,
        "conv_percent": conv_percent,
    }

    # is the suggested point outside the grid?
    if next_step["Nb"] > bnd_max or next_step["Gc"] > cut_max:
        next_step["new_grid"] = True

    # was the suggested point already computed?
    if (next_step["Nb"] in grid_plot[:, 0]) and (
        next_step["Gc"]
        in grid_plot[np.where(grid_plot[:, 0] == next_step["Nb"]), 1]
    ):
        next_step["already_computed"] = True
        temp_idx = (grid_plot[:, 0] == next_step["Nb"]) & (
            grid_plot[:, 1] == next_step["Gc"]
         )
        next_step["already_computed_gap"] = grid_plot[temp_idx][0][2]

    # messages
    print("\nNext step data:", flush=True)
    print(next_step, flush=True)

    return next_step

Fit the first grid.

In [8]:
fit_flag, mae_fit, x_fit, y_fit, z_fit, extra, alpha, beta = get_best_fit(grids[0], conv_grad, conv_hessian)
if fit_flag:
    print(f"\nWorking fit was found with: alpha = {alpha:d}, beta = {beta:d}!")
else:
    print("\nFitting failed...")


Trying: alpha = 1.00, beta = 1.00
Number of points that fullfil the derivative criteria: 0
MAE = 0.025478

Trying: alpha = 1.00, beta = 2.00
Number of points that fullfil the derivative criteria: 740
MAE = 0.042995

Trying: alpha = 2.00, beta = 1.00
Number of points that fullfil the derivative criteria: 0
MAE = 0.025339

Trying: alpha = 2.00, beta = 2.00
Number of points that fullfil the derivative criteria: 760
MAE = 0.041516

Best power law: alpha = 2.00, beta = 1.00

Number of points that fullfil the derivative criteria: 0

Fitting failed...


Fit the second grid.

In [9]:
fit_flag, mae_fit, x_fit, y_fit, z_fit, extra, alpha, beta = get_best_fit(grids[1], conv_grad, conv_hessian)
if fit_flag:
    print(f"\nWorking fit was found with: alpha = {alpha:d}, beta = {beta:d}!")
else:
    print("\nFitting failed...")


Trying: alpha = 1.00, beta = 1.00
Number of points that fullfil the derivative criteria: 0
MAE = 0.012784

Trying: alpha = 1.00, beta = 2.00
Number of points that fullfil the derivative criteria: 1128
MAE = 0.008738

Trying: alpha = 2.00, beta = 1.00
Number of points that fullfil the derivative criteria: 0
MAE = 0.012607

Trying: alpha = 2.00, beta = 2.00
Number of points that fullfil the derivative criteria: 1128
MAE = 0.008878

Best power law: alpha = 1.00, beta = 2.00

Number of points that fullfil the derivative criteria: 1128

Working fit was found with: alpha = 1, beta = 2!


Check where the next calculation should be done.

In [10]:
next_point = suggest_next_point(x_fit, y_fit, z_fit, extra, conv_thr)

Extrapolated gap value from fit parameters: 7.595807 eV
Extrapolated gap value at points that fulfill the derivative criteria: 7.580713 eV

Next step data:
{'Nb': 1000.0, 'Gc': 24.0, 'gap': 7.533819224190457, 'ref_gap': 7.580713455884243, 'new_grid': False, 'already_computed': False, 'conv_thr': 0.082, 'conv_percent': 0}


This does not agree with the results of the publication.
After reviewing the original code, we found that the first derivatives were
were defined incorrectly. So we try the fitting procedure again with the wrong first derivatives in the fitting function.

In [11]:
def fit2d(grid, conv_grad, conv_hessian, alpha=1, beta=1):
    # fit functions for the convergence surface
    f = lambda x, a, b, c, d: (a / (x[0] ** alpha) + b) * (c / (x[1] ** beta) + d)

    # wrong definitions of the fit function derivatives from the aiida yambo repository
    fx = lambda x, a, c, d: (
        (-alpha * a / (x[0] ** (alpha + 1))) * (c / (x[1]) + d)
    )
    fy = lambda x, a, b, c: (
        (a / (x[0]) + b) * (-beta * c / (x[1] ** (beta + 1)))
    )
    
    """
    # corrected definitions of the fit function derivatives
    fx = lambda x, a, c, d: (
        (-alpha * a / (x[0] ** (alpha + 1))) * (c / (x[1] ** beta) + d)
    )
    fy = lambda x, a, b, c: (
        (a / (x[0] ** alpha) + b) * (-beta * c / (x[1] ** (beta + 1)))
    )
    """

    # off-diagonal elements of the hessian matrix of the fit function
    fxy = lambda x, a, c: (
        (-alpha * a / (x[0] ** (alpha + 1))) * (-beta * c / (x[1] ** (beta + 1)))
    )

    # fit data
    xdata, ydata = (
        np.array((grid[:, 0], grid[:, 1])),
        grid[:, 2],
    )

    # fit the convergence surface
    popt, _ = scipy.optimize.curve_fit(
        f,
        xdata=xdata,
        ydata=ydata,
        sigma=1 / (xdata[0] * xdata[1]),
        bounds=(
            [-np.inf, -np.inf, -np.inf, -np.inf],
            [np.inf, np.inf, np.inf, np.inf],
        ),
    )

    # convergence critierum
    mae_int = np.average(
        (
            abs(
                f(
                    xdata,
                    popt[0],
                    popt[1],
                    popt[2],
                    popt[3],
                )
                - ydata
            )
        ),
        weights=xdata[0] * xdata[1],
    )
    mae_fit = mae_int

    # get the extrapolated gap value
    extra = popt[1] * popt[3]

    # new grid to extrapolate the function far from the starting grid
    x_fit = np.arange(min(xdata[0]), max(xdata[0]) * 10, bnd_step)
    y_fit = np.arange(min(xdata[1]), max(xdata[1]) * 10, cut_step)

    # obtain the derivatives
    zx_fit = fx(np.meshgrid(x_fit, y_fit), popt[0], popt[2], popt[3])
    zy_fit = fy(np.meshgrid(x_fit, y_fit), popt[0], popt[1], popt[2])
    zxy_fit = fxy(np.meshgrid(x_fit, y_fit), popt[0], popt[2])

    # get the fit function values on the new grid
    x_fit, y_fit = np.meshgrid(x_fit, y_fit)
    z_fit = f(
        np.meshgrid(x_fit, y_fit), popt[0], popt[1], popt[2], popt[3]
    )

    # check where the convergence conditions for the derivatives are meet
    condition_conv_calc = np.where(
        (np.abs(zx_fit) < conv_grad)
        & (np.abs(zy_fit) < conv_grad)
        & (np.abs(zxy_fit) < conv_hessian)
    )

    # save this fit to the old fit variable for later
    old_x_fit = x_fit
    old_y_fit = y_fit
    old_z_fit = z_fit

    # if no points match the convergence critieria return
    print(
        "Number of points that fullfil the derivative criteria: "
        + f"{len(x_fit[condition_conv_calc]):d}",
        flush=True,
    )

    if len(x_fit[condition_conv_calc]) == 0:
        return False, mae_fit, x_fit, y_fit, z_fit, extra

    # obtain a new grid where the estimated converged direct gap is situated
    b = max(max(xdata[0]), x_fit[condition_conv_calc][0] * 1.25)
    g = max(max(xdata[1]), y_fit[condition_conv_calc][0] * 1.25)

    # fit on the new grid with derivatives
    x_fit = np.arange(min(xdata[0]), b + 1, bnd_step)
    y_fit = np.arange(min(xdata[1]), g + 1, cut_step)
    z_fit = f(
        np.meshgrid(x_fit, y_fit), popt[0], popt[1], popt[2], popt[3]
    )
    zx_fit = fx(np.meshgrid(x_fit, y_fit), popt[0], popt[2], popt[3])
    zy_fit = fy(np.meshgrid(x_fit, y_fit), popt[0], popt[1], popt[2])
    zxy_fit = fxy(np.meshgrid(x_fit, y_fit), popt[0], popt[2])
    x_fit, y_fit = np.meshgrid(x_fit, y_fit)

    return True, mae_fit, x_fit, y_fit, z_fit, extra

def get_best_fit(grid, conv_grad, conv_hessian, power_laws=[1, 2]):
    # find the best initial fit parameters for the exponents
    error = 10  # random initial value
    for i in power_laws:
        for j in power_laws:
            print(f"\nTrying: alpha = {i:.2f}, beta = {j:.2f}", flush=True)
            fit_flag, mae_fit, x_fit, y_fit, z_fit, extra = fit2d(
                grid,
                conv_grad,
                conv_hessian,
                alpha=i,
                beta=j,
            )
            print(f"MAE = {mae_fit:.6f}", flush=True)
            if mae_fit < error:
                ii, jj = i, j
                error = mae_fit
    print(f"\nBest power law: alpha = {ii:.2f}, beta = {jj:.2f}\n", flush=True)

    # get the best initial fit
    fit_flag, mae_fit, x_fit, y_fit, z_fit, extra = fit2d(
        grid,
        conv_grad,
        conv_hessian,
        alpha=ii,
        beta=jj,
    )
    
    return fit_flag, mae_fit, x_fit, y_fit, z_fit, extra, ii, jj

def suggest_next_point(x_fit, y_fit, z_fit, extra, conv_thr, conv_percent=0):
    # reference
    reference = z_fit[-1, -1]
    print(
        "Extrapolated gap value from fit parameters: " + f"{extra:.6f} eV",
        flush=True,
    )
    print(
        "Extrapolated gap value at points that fulfill the derivative criteria: "
        + f"{reference:.6f} eV",
        flush=True,
    )

    # converge with percentages?
    if conv_percent > 0:
        conv_thr = conv_percent / 100 * reference

    # find a point that satisfies the convergence condition
    discrepancy = np.round(
        abs(reference - z_fit),
        abs(int(np.round(np.log10(conv_thr), 0))),
    )
    condition = np.where((discrepancy <= conv_thr))
    next_bnd_x, next_cutsrc = (
        x_fit[condition][0],
        y_fit[condition][0],
    )
    ref_gap = z_fit[condition][0]

    # dictionary that contains the relevant information for the next step
    next_step = {
        "Nb": next_bnd_x,
        "Gc": next_cutsrc,
        "gap": ref_gap,
        "ref_gap": reference,
        "new_grid": False,
        "already_computed": False,
        "conv_thr": conv_thr,
        "conv_percent": conv_percent,
    }

    # is the suggested point outside the grid?
    if next_step["Nb"] > bnd_max or next_step["Gc"] > cut_max:
        next_step["new_grid"] = True

    # was the suggested point already computed?
    if (next_step["Nb"] in grid_plot[:, 0]) and (
        next_step["Gc"]
        in grid_plot[np.where(grid_plot[:, 0] == next_step["Nb"]), 1]
    ):
        next_step["already_computed"] = True
        temp_idx = (grid_plot[:, 0] == next_step["Nb"]) & (
            grid_plot[:, 1] == next_step["Gc"]
         )
        next_step["already_computed_gap"] = grid_plot[temp_idx][0][2]

    # messages
    print("\nNext step data:", flush=True)
    print(next_step, flush=True)

    return next_step

In [12]:
fit_flag, mae_fit, x_fit, y_fit, z_fit, extra, alpha, beta = get_best_fit(grids[1], conv_grad, conv_hessian)
if fit_flag:
    print(f"\nWorking fit was found with: alpha = {alpha:d}, beta = {beta:d}!")
else:
    print("\nFitting failed...")


Trying: alpha = 1.00, beta = 1.00
Number of points that fullfil the derivative criteria: 0
MAE = 0.012784

Trying: alpha = 1.00, beta = 2.00
Number of points that fullfil the derivative criteria: 1128
MAE = 0.008738

Trying: alpha = 2.00, beta = 1.00
Number of points that fullfil the derivative criteria: 3
MAE = 0.012607

Trying: alpha = 2.00, beta = 2.00
Number of points that fullfil the derivative criteria: 1170
MAE = 0.008878

Best power law: alpha = 1.00, beta = 2.00

Number of points that fullfil the derivative criteria: 1128

Working fit was found with: alpha = 1, beta = 2!


Check where the next calculation should be done.

In [13]:
next_point = suggest_next_point(x_fit, y_fit, z_fit, extra, conv_thr)

Extrapolated gap value from fit parameters: 7.595807 eV
Extrapolated gap value at points that fulfill the derivative criteria: 7.580713 eV

Next step data:
{'Nb': 1000.0, 'Gc': 24.0, 'gap': 7.533819224190457, 'ref_gap': 7.580713455884243, 'new_grid': False, 'already_computed': False, 'conv_thr': 0.082, 'conv_percent': 0}


This also does not match the result shown in the publication. 